In [2]:
from localmd.decomposition import localmd_decomposition, display, factored_svd, windowed_pmd, single_residual_block_md, get_projector
from localmd.dataset import MultipageTiffDataset
import localmd.visualization 
from localmd.visualization import generate_PMD_comparison_triptych

import scipy
import scipy.sparse
import jax
import jax.scipy
import jax.numpy as jnp
from jax import jit, vmap
import numpy as np
import math

import os
import tifffile

%load_ext autoreload

# Specify the dataset in the below panel. See PMD documentation for easy ways to add support for your custom data formats. We provide default support for single, multipage tiff files. 

In [2]:
input_file = "../datasets/demoMovie.tif"
current_dataset = MultipageTiffDataset(input_file)

# Below panel specifies the parameters for running the method

In [7]:
#Default parameters (can be modified..)

pmd_params_dict = {
    'block_height':20,
    'block_width':20,
    'frames_to_init':5000,
    'background_rank':1,
    'max_consec_failures':1,
    'max_components':40,
}


#NOTE: this data folder will also contain the location of the TestData
block_height = pmd_params_dict['block_height']
block_width = pmd_params_dict['block_width'] 

block_sizes = [block_height, block_width]

max_consec_failures = pmd_params_dict['max_consec_failures']
frames_to_init = pmd_params_dict['frames_to_init']
background_rank = pmd_params_dict['background_rank'] 

###THESE PARAMS ARE NEVER MODIFIED
sim_conf = 5

max_components = pmd_params_dict['max_components']

corrector = None

frame_batch_size = 2000
pixel_batch_size = 10000
dtype="float32"

order = "F"

# Run PMD

In [1]:
U, R, s, V, std_img, mean_img, data_shape, data_order = localmd_decomposition(current_dataset, block_sizes, frames_to_init, \
                                max_components=max_components, background_rank = background_rank, sim_conf=sim_conf,\
                                 frame_batch_size=frame_batch_size,pixel_batch_size=pixel_batch_size, dtype=dtype, order=order, \
                                 num_workers=0, frame_corrector_obj = corrector, max_consec_failures=max_consec_failures)

# Save the compressed results into a NPZ file

In [6]:
npz_save_name = "INSERT_SAVE_NAME_HERE.npz"
U = U.tocsr()
np.savez(npz_save_name, fov_shape = data_shape[:2], \
                fov_order=data_order, U_data = U.data, \
                U_indices = U.indices,\
                U_indptr=U.indptr, \
                U_shape = U.shape, \
                U_format = type(U), \
                R = R, \
                s = s, \
                Vt = V, \
                 mean_img = mean_img, \
                 noise_var_img = std_img)

# Generate a comparison triptych to show how well PMD retains signal from the original movie

In [9]:
########### MODIFY ENCLOSED VARIABLES AS DESIRED:

#These two intervals specify what part of the FOV we want to analyze. The below parameters mean that we will crop the FOV at [0, 100] x [0, 100] 
# when generating the comparison video
dim1_interval = [0, 100]
dim2_interval = [0, 100] 

#Specify which frames you want to see
start_frame = 0
end_frame = 2000
frames = [i for i in range(start_frame, end_frame)]
############# 



output_triptych = generate_PMD_comparison_triptych(current_dataset, frames, U, R, s, V, mean_img, std_img, data_order, data_shape,\
                                 dim1_interval, dim2_interval, frame_corrector=None)

# Save the comparison triptych as a multipage tiff file for easy viewing in imageJ

In [10]:
## Save the triptych as a tiff file, which can be viewed in imageJ

## Modify the filename below as desired
filename_to_save = "Denoised_Vs_Raw_Comparison.tiff"

#The below line saves the tiff file
tifffile.imwrite(filename_to_save, output_triptych.transpose(2, 0, 1).astype("float32"))